In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_squared_error
from scipy import stats
import numpy as np

## Load the dataset

In [ ]:
# Source of data: http://web.stanford.edu/~hastie/Papers/LARS/LeastAngle_2002.pdf
#
# AGE SEX BMI BP ··· Serum Measurements ··· Response
# x1  x2  x3  x4     x5 x6 x7 x8 x9 x10     y

diabetes = datasets.load_diabetes()
X = diabetes.data
y = diabetes.target
print('dim(X) = ', X.shape)
np.set_printoptions(precision=3)
print('X=\n', X)

dim(X) =  (442, 10)
X=
 [[ 0.038  0.051  0.062 ... -0.003  0.02  -0.018]
 [-0.002 -0.045 -0.051 ... -0.039 -0.068 -0.092]
 [ 0.085  0.051  0.044 ... -0.003  0.003 -0.026]
 ...
 [ 0.042  0.051 -0.016 ... -0.011 -0.047  0.015]
 [-0.045 -0.045  0.039 ...  0.027  0.045 -0.026]
 [-0.045 -0.045 -0.073 ... -0.039 -0.004  0.003]]


In [ ]:
# split the dataset into tran (2/3) and test (1/3)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
print('dim(X_train) = ', X_train.shape)
print('dim(X_test) = ', X_test.shape)

dim(X_train) =  (296, 10)
dim(X_test) =  (146, 10)


## Ridge regression with LOOCV

In [ ]:
# Leave One Out Cross Validation
l2_cv = RidgeCV(cv=None, store_cv_values=True)
l2_cv.fit(X_train, y_train)
print('alpha = ', l2_cv.alpha_)
print('coef = ', l2_cv.coef_)

alpha =  0.1
coef =  [  39.096 -197.812  511.287  358.842  -93.211  -73.24  -220.198  118.041
  344.885   25.161]


In [ ]:
# What are the regularization parameters tried in the RidgeCV?
print(l2_cv.alphas)

[ 0.1  1.  10. ]


In [ ]:
# Print the individual performance scores for the 10 folds coresponding to all regularization coefficients
# by default cv_values store the mean squared errors
print('dim(cv_values)=', l2_cv.cv_values_.shape)
print('cv_values=\n', l2_cv.cv_values_[:5,:])

dim(cv_values)= (296, 3)
cv_values=
 [[2.366e+00 5.865e+00 4.958e+00]
 [1.082e+02 4.174e+00 8.459e+02]
 [3.313e+02 1.575e+03 1.670e+03]
 [4.632e+03 7.043e+03 5.938e+03]
 [2.334e+03 2.399e+03 1.339e+03]]


In [ ]:
# Get the mean and standard error of the performance metric - by default is the mean squared errors
means_MSE = l2_cv.cv_values_.mean(axis=0)
SEs_MSE = l2_cv.cv_values_.std(axis=0)/np.sqrt(l2_cv.cv_values_.shape[0])
print('means = ', means_MSE)
print('SEs = ', SEs_MSE)

means =  [3159.13  3675.778 5274.707]
SEs =  [229.249 241.515 339.414]


In [ ]:
# compare the average prediction error given by CV
# with the testing error on the testing dataset
print('95% conf. int. avg. pred. MSE (alpha=0.1) = [', means_MSE[0]-1.96*SEs_MSE[0],
      ', ', means_MSE[0]+1.96*SEs_MSE[0], ']')

y_pred_test = l2_cv.predict(X_test)
test_RMSE = mean_squared_error(y_test, y_pred_test)
print('testing error obtained in testing dataset = ', test_RMSE)

95% conf. int. avg. pred. MSE (alpha=0.1) = [ 2709.802711995863 ,  3608.4568395757183 ]
testing error obtained in testing dataset =  2807.3890485641673


### **Change example_Ridge.ipynb by copying which_is_better() function from Hypothesis_Testing.ipynb.**

In [ ]:
def which_is_better(M1_values, M2_values, alpha_one_tail=0.025):
    # Test whether mean(M1) > mean(M2) using Welch's t-test
    # at default alpha = 0.025 significance level for the one tail test
    #
    # Note that stats.ttest_ind return the two-tailed p-value
    #
    # Null hypothesis: mean(M1) <= mean(M2)
    # Alternative: mean(M1) > mean(M2)
    #
    # REJECT if pvalue/2 < alpha

    tstat, pvalue = stats.ttest_ind(M1_values, M2_values, equal_var = False)
    print('tstat = ', tstat)
    print('pvalue = ', pvalue)
    if pvalue/2 < alpha_one_tail:
        if tstat < 0.0:
            print('Null rejected: more confident that mean(M2) > mean(M1)')
        else:
            print('Null rejected: more confident that mean(M1) > mean(M2)')
        return True
    else:
        print('Cannot reject null: no confidence which one is better')
        return False

**1. Which model do you prefer between alpha=0.1 and alpha=1?**

In [ ]:
which_is_better(l2_cv.cv_values_[:,0],l2_cv.cv_values_[:,1])

tstat =  -1.5489027377020113
pvalue =  0.1219427641388961
Cannot reject null: no confidence which one is better


False

**2. Which model do you prefer between alpha=0.1 and alpha=10?**


In [ ]:
which_is_better(l2_cv.cv_values_[:,0],l2_cv.cv_values_[:,2])

tstat =  -5.156485694998025
pvalue =  3.587824628329308e-07
Null rejected: more confident that mean(M2) > mean(M1)


True

**3. Which model do you prefer between alpha=1 and alpha=10?**

In [ ]:
which_is_better(l2_cv.cv_values_[:,1],l2_cv.cv_values_[:,2])

tstat =  -3.8318182953512436
pvalue =  0.00014236470562314335
Null rejected: more confident that mean(M2) > mean(M1)


True

**4. Now given that larger alphas correspond to simple models, which model do you prefer from these three models?**

**Answer :**Given that, **larger alpha values are correspond to simple model** then we can prefere the model which has samller mean value as performace matric, this gives us better performance.

According to above code we get greater mean value is for alpha= 10

so, we can write **mean(alpha=10) > mean(alpha=0.1)**

**Also, mean(alpha=10) > mean(alpha=1)**

Welch's t-test for mean(alpha=0.1) and  mean(alpha=1) is: Cannot reject null: no confidence which one is better.

so, the mean(alpha=10) and mean(alpha=1) is equal  using Welch's t-test.

For that reason we are selecting model with **alpha value is 1** .

**5. Is the model selected by the hypothesis testing different from the model selected by RidgeCV?**

**Answer:** The model selected by hypothesis testing is **Different** from as the model selected by RidgeCV